In [1]:
import pandas as pd

# 策略績效指標（示範）
df = pd.DataFrame({
    "strategy": ["S1", "S2", "S3", "S4", "S5"],
    "sharpe": [1.2, 0.8, 1.5, 0.6, 1.1],
    "winrate": [0.65, 0.55, 0.40, 0.70, 0.50],
    "maxdd": [-0.12, -0.08, -0.18, -0.10, -0.15],      # 越小越好
    "recovery_days": [15, 28, 40, 22, 33],             # 越小越好
    "cagr": [0.10, 0.07, 0.12, 0.06, 0.09]
})
df

,strategy,sharpe,winrate,maxdd,recovery_days,cagr
0,S1,1.2,0.65,-0.12,15,0.10
1,S2,0.8,0.55,-0.08,28,0.07
2,S3,1.5,0.40,-0.18,40,0.12
3,S4,0.6,0.70,-0.10,22,0.06
4,S5,1.1,0.50,-0.15,33,0.09


In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import pandas as pd
import numpy as np

def pca_strategy_score_plotly(df, metric_cols, reverse_cols=None, verbose=True):
    """
    用 PCA 幫策略做綜合評分（Plotly 版本）
    """
    df = df.copy()
    reverse_cols = reverse_cols or []

    # 1️⃣ 統一方向（越大越好）
    adj_cols = []
    for col in metric_cols:
        if col in reverse_cols:
            new_col = col + "_adj"
            df[new_col] = -df[col]
            adj_cols.append(new_col)
        else:
            adj_cols.append(col)

    # 2️⃣ 標準化 Z-score
    scaler = StandardScaler()
    Z = scaler.fit_transform(df[adj_cols])

    # 3️⃣ PCA
    pca = PCA(n_components=1)
    PC1 = pca.fit_transform(Z)

    df["pca_score"] = PC1
    df["rank"] = df["pca_score"].rank(ascending=False)

    loadings = pd.Series(pca.components_[0], index=adj_cols)
    var_ratio = pca.explained_variance_ratio_[0]

    if verbose:
        print("\n=== PCA 分數 ===")
        print(df[["strategy", "pca_score", "rank"]].sort_values("rank"))
        print("\n=== Loading（指標權重）===")
        print(loadings)
        print("\nPC1 說明變異量:", var_ratio)

    return df, loadings, var_ratio

In [4]:
def plot_loadings_plotly(loadings):
    fig = go.Figure(go.Bar(
        x=loadings.index,
        y=loadings.values,
        text=[f"{v:.3f}" for v in loadings.values],
        textposition="auto"
    ))
    fig.update_layout(
        title="PCA Loadings（指標重要程度）",
        xaxis_title="指標",
        yaxis_title="Loading Weight",
        template="plotly_white"
    )
    fig.show()

In [5]:
def plot_variance_plotly(var_ratio):
    fig = go.Figure(go.Bar(
        x=["PC1"],
        y=[var_ratio],
        text=[f"{var_ratio:.3f}"],
        textposition="auto"
    ))
    fig.update_layout(
        title="Explained Variance Ratio",
        xaxis_title="Principal Component",
        yaxis_title="Variance %",
        template="plotly_white"
    )
    fig.show()

In [6]:
metrics = ["sharpe", "winrate", "maxdd", "recovery_days", "cagr"]
neg = ["maxdd", "recovery_days"]

result_df, loadings, var_ratio = pca_strategy_score_plotly(
    df, metric_cols=metrics, reverse_cols=neg
)

plot_loadings_plotly(loadings)
plot_variance_plotly(var_ratio)


=== PCA 分數 ===
  strategy  pca_score  rank
2       S3   3.313464   1.0
4       S5   0.944701   2.0
0       S1  -0.534486   3.0
1       S2  -1.282539   4.0
3       S4  -2.441141   5.0

=== Loading（指標權重）===
sharpe               0.464161
winrate             -0.456829
maxdd_adj            0.465825
recovery_days_adj   -0.387054
cagr                 0.457230
dtype: float64

PC1 說明變異量: 0.7904502215140607
